In [ ]:
from pyfreefem import FreeFemRunner, readFFArray
from nullspace_optimizer import Optimizable, nlspace_solve
from pymedit import Mesh, P1Function, trunc, Mesh3D, cube, mmg3d, generate3DMesh, P1Function3D, trunc3DMesh
from pymedit import saveToVtk, advect, P1Vector3D, mshdist
import numpy as np
import shutil
import pandas as pd

output='output/caseCylinderTest' #1
import os
os.makedirs(output,exist_ok=True)

config={'OUTPUT':output}

N=8 #number of cores
i = format(0,'04d')
meshes='output'
M = Mesh3D(meshes+"/ThCoarse.mesh")
M.save(output+"/Th00.mesh")

preamble="""
func int readSolFile(mesh & Th, string fileName, real[int] & phi){
    ifstream f(fileName);
    string dummy="";
    while(dummy(0:2)!="Sol"){
        f>>dummy;
    }
    int n;
    f >> n;
    if(n!=Th.nv){
        cout << "Error : the number of vertices in the file "+fileName+" does not correspond to the mesh in memory"<< endl;
        exit(1);
    }
    f >> dummy;
    f >> dummy;
    for(int i=0;i<Th.nv;i++){
        f>>phi[i];
    }
}

func int saveArray(string fileName, real[int] &value){
    ofstream file(fileName);
    file.precision(16);
    file << value;
}

func int readData(string fileName, real[int] &data){
    {
        ifstream f(fileName);
        f>>data;
    }
}

include "macros.edp"
load "medit"
load "msh3"
load "iovtk"

int region1 = 20; //interior circle
int region2 = 3; //exterior donuts
int empty = 2;
real k1=0.15;
real k2=0.1; //0.1*1e-6; //thermal diffusivity D or alpha of rubber;
real rhocp=1e2;
real vtarget=0.0942477796; 
real nu = 1e-2; //1e-3; //1e-6
real beta=2e1;//2e1; //1e2
"""

mesh_code='''
load "PETSc"
macro dimension()3 //EOM
include "macro_ddm.idp"

mesh3 ThBox=readmesh3("$OUTPUT/Th00.mesh");
savevtk("$OUTPUT/ThBox.vtu",ThBox);
mesh3 Th=trunc(ThBox, ((region == region1) || (region == region2)));
savemesh(Th,"$OUTPUT/Th.mesh");
{
    ofstream f("$OUTPUT/V.gp");
    f.precision(16);
    f << int3d(Th,region2,qforder=1)(1.) - vtarget  << endl;
}
'''

solve_kappa = """
mesh3 ThGlobal = Th;

int[int] n2o;
macro ThN2O() n2o // this tells buildDmesh to keep the local to global correspondence
buildDmesh(Th)

fespace PhGlobal(ThGlobal, P1);

PhGlobal kappaGlobal, kappasum, dOmegaGlobal, nx, ny, nz;

if (mpirank==0){
    //save the solution
    kappasum = 1./0.1;
    savesol("$OUTPUT/kappa.sol",ThGlobal,kappasum);
}


"""

interpolate_ns="""
                mesh3 ThBox=readmesh3("$OUTPUT/Th.mesh");
                mesh3 Th=trunc(ThBox, ((region == region1) || (region == region2)));
                mesh3 Th1=trunc(Th, (region == region1));
                savemesh(Th,"$OUTPUT/Th.mesh");

                fespace Phnew(Th1, [P2,P2,P2,P1]);
                Phnew [uxnew,uynew,uznew,pnew];

                [uxnew,uynew,uznew,pnew] = [(0.1^2 - (y-0.5)^2 - (z-0.5)^2)/0.01,0.,0.,0.];
                savesol("$OUTPUT/ux.sol",Th1,uxnew);
                savesol("$OUTPUT/uy.sol",Th1,uynew);
                savesol("$OUTPUT/uz.sol",Th1,uznew);
            """


def computeU(mesh,t):
    FreeFemRunner([preamble, mesh_code, solve_kappa],config,run_dir=output,run_file='solve.edp',debug=1).execute({'MESH':mesh},ncpu=N)
    FreeFemRunner([preamble, interpolate_ns],config,run_dir=output,run_file='interpolate_ns.edp',debug=1).execute({'MESH':mesh})
    os.system(f"time mpiexec -n 1 ./MatrixBinaryPetscTP2RealValues {output} {t} -pc_type lu -pc_factor_mat_solver_type superlu_dist -ksp_monitor ")
    
    with open(output+'/J.gp','r') as f:
        J = float(f.readlines()[0])
    with open(output+'/V.gp','r') as f:
        V = float(f.readlines()[0])
    return (J,V)

code_sensitivity=r"""
load "PETSc"
macro dimension()3 //EOM
include "macro_ddm.idp"

mesh3 ThBox=readmesh3("$MESH");
mesh3 ThBoxGlobal = ThBox;
mesh3 Th2 = readmesh3("$OUTPUT/Th2.mesh");

int[int] n2oBox;
macro ThBoxN2O() n2oBox // this tells buildDmesh to keep the local to global correspondence
buildDmesh(ThBox)

// Variational space
fespace Ph2(Th2, P1);
fespace PhBox(ThBoxGlobal, P1);

PhBox dOmegaBox,kappaP1,nx3,ny3,nz3,thetax2,thetay2,thetaz2;
Ph2 T2,R2,T0;

T2[] = readsol("$OUTPUT/T2.sol");
R2[] = readsol("$OUTPUT/R2.sol");
T0 = 0.;

if (mpirank==0){
    int[int] Order = [1];
    string DataName = "T";
    savevtk("$OUTPUT/T.vtu",Th2,T2,dataname=DataName, order=Order);
    //savevtk("$OUTPUT/R.vtu",Th2,R2);
}

if (mpirank==0){
    nx3 = 0.; ny3=(y-0.5)/sqrt((y-0.5)^2+(z-0.5)^2); nz3 = (z-0.5)/sqrt((y-0.5)^2+(z-0.5)^2);
    thetax2 = -nx3; thetay2 = -ny3; thetaz2 = -nz3;
    kappaP1 = 1./sqrt((y-0.5)^2+(z-0.5)^2);

    PhBox  dxthetax, dythetax, dzthetax, dxthetay, dythetay, dzthetay, dxthetaz, dythetaz, dzthetaz;

    dxthetax = 0.;
    dythetax = 0.;
    dzthetax = 0.;
    
    dxthetay = 0.;
    dythetay = -(z-0.5)^2/((y-0.5)^2 + (z-0.5)^2)^1.5;
    dzthetay = ((y-0.5)*(z-0.5))/((y-0.5)^2 + (z-0.5)^2)^1.5;
    
    dxthetaz = 0.;
    dythetaz = ((y-0.5)*(z-0.5))/((y-0.5)^2 + (z-0.5)^2)^1.5;
    dzthetaz = -(y-0.5)^2/((y-0.5)^2 + (z-0.5)^2)^1.5;
    
    //surface
    real val1 = int2d(ThBoxGlobal,10,qforder=3)((thetax2*nx3+thetay2*ny3+thetaz2*nz3)
                                          *(beta^2*T2^2*(kappaP1 - 4.*beta/k2) + beta*T2*R2*(2.*beta/k2 - kappaP1) - k2*tr(grad(T2))*grad(R2) ) );
    

    //volume
    real val2 = int2d(ThBoxGlobal,10,qforder=3)((dxthetax+dythetay+dzthetaz - tr(MatrixByVector([dxthetax,dythetax,dzthetax,
                                                                                       dxthetay,dythetay,dzthetay,
                                                                                       dxthetaz,dythetaz,dzthetaz],[nx3,ny3,nz3]))*[nx3,ny3,nz3] )*(beta^2*T2^2 - beta*T2*R2))
                +int3d(ThBoxGlobal,region2,qforder=3)(-(dxthetax+dythetay+dzthetaz)*k2*tr(grad(T2))*grad(R2) + k2*tr(grad(R2))*MatrixByVector([2.*dxthetax,dxthetay+dythetax,dxthetaz+dzthetax,
                                                                                                                                     dythetax+dxthetay, 2.*dythetay,dythetaz+dzthetay,
                                                                                                                                     dzthetax+dxthetaz,dzthetay+dythetaz,2.*dzthetaz], grad(T2)));
                                                                                                                                     
    {
        ofstream f("$OUTPUT/val1.gp");
        f.precision(16);
        f << val1  << endl;
    }
    {
        ofstream f("$OUTPUT/val2.gp");
        f.precision(16);
        f << val2  << endl;
    }
}

"""
def sensitivity(mesh):
    FreeFemRunner([preamble,code_sensitivity],config,run_dir=output,run_file='sensitivities.edp',debug=1).execute(config={'MESH':mesh},ncpu=N)

    with open(output+'/val1.gp','r') as f:
        val1 = float(f.readlines()[0])
    with open(output+'/val2.gp','r') as f:
        val2 = float(f.readlines()[0])
    return (val1,val2)

import matplotlib.pyplot as plt

## Initial solution and J ##
mesh0 = output+"/Th00.mesh"
(J,V) = computeU(mesh0,0)
(val1,val2) = sensitivity(mesh0)

T = 10.**np.arange(-3,-0.8,0.2)
delta1 = []
delta2 = []
for t in T:
    (Jt,Vt) = computeU(mesh0,t)

    d1 = np.abs(Jt - J - t*val1)
    print(f"t={np.log10(t)},Jt={Jt},J={J},Jt-J={Jt-J},der={t*val1},delta1={np.log10(d1)}")
    delta1.append(d1)

    open(output+'/delta1.txt', 'w').close()
    df = pd.DataFrame(data=np.array(delta1),columns=['delta1'])
    df.to_csv(output+'/delta1.txt', header=None, index=None, sep=' ', mode='a')


    d2 = np.abs(Jt - J - t*val2)
    print(f"t={np.log10(t)},Jt={Jt},J={J},Jt-J={Jt-J},der={t*val2},delta2={np.log10(d2)}")
    delta2.append(d2)

    open(output+'/delta2.txt', 'w').close()
    df = pd.DataFrame(data=np.array(delta2),columns=['delta2'])
    df.to_csv(output+'/delta2.txt', header=None, index=None, sep=' ', mode='a')

ff-mpirun -np 8 output/caseCylinderTest/solve.edp -v -1 -nw (19.98s)
FreeFem++ output/caseCylinderTest/interpolate_ns.edp -v -1 -nw (8.99s)
tau = 0, thetay(1,1,1) = -0
  0 KSP Residual norm 1.168579237596e+04
  1 KSP Residual norm 2.918983915018e-10
Text = 0, int2d = 146.166
  0 KSP Residual norm 4.777742835881e+03
  1 KSP Residual norm 9.741798663620e-11
ff-mpirun -np 8 output/caseCylinderTest/sensitivities.edp -v -1 -nw

6446.46user 10.89system 2:14:06elapsed 80%CPU (0avgtext+0avgdata 6956980maxresident)k
52040inputs+71456outputs (2910major+2741973minor)pagefaults 0swaps


 (42.08s)
ff-mpirun -np 8 output/caseCylinderTest/solve.edp -v -1 -nw (24.09s)
FreeFem++ output/caseCylinderTest/interpolate_ns.edp -v -1 -nw (9.94s)
tau = 0.001, thetay(1,1,1) = -0.000707107
  0 KSP Residual norm 1.169448984470e+04
  1 KSP Residual norm 1.195070117386e-10
Text = 0, int2d = 145.332
  0 KSP Residual norm 4.772361309407e+03
  1 KSP Residual norm 5.630513579576e-11
t=-3.0,Jt=145.33228,J=146.16566,Jt-J=-0.8333800000000053,der=1.086556615696084,delta1=0.2832868912515627
t=-3.0,Jt=145.33228,J=146.16566,Jt-J=-0.8333800000000053,der=-0.8307361999499322,delta2=-2.577771393525862
ff-mpirun -np 8 output/caseCylinderTest/solve.edp -v -1 -nw

4708.63user 8.91system 1:26:06elapsed 91%CPU (0avgtext+0avgdata 6963048maxresident)k
0inputs+71448outputs (1major+2739347minor)pagefaults 0swaps


 (22.99s)
FreeFem++ output/caseCylinderTest/interpolate_ns.edp -v -1 -nw (8.19s)
tau = 0.00158489, thetay(1,1,1) = -0.00112069
  0 KSP Residual norm 1.169957955910e+04
  1 KSP Residual norm 1.008721776986e-10
Text = 0, int2d = 144.843
  0 KSP Residual norm 4.769155159115e+03
  1 KSP Residual norm 5.788091295034e-11
t=-2.8,Jt=144.84256,J=146.16566,Jt-J=-1.3231000000000108,der=1.7220761834403104,delta1=0.4836124244836494
t=-2.8,Jt=144.84256,J=146.16566,Jt-J=-1.3231000000000108,der=-1.3166281480316624,delta2=-2.1889714249657266
ff-mpirun -np 8 output/caseCylinderTest/solve.edp -v -1 -nw

...

 (29.67s)
FreeFem++ output/caseCylinderTest/interpolate_ns.edp -v -1 -nw (18.48s)
tau = 0.01, thetay(1,1,1) = -0.00707107
  0 KSP Residual norm 1.177295093748e+04
  1 KSP Residual norm 1.687937308679e-10
Text = 0, int2d = 137.604
  0 KSP Residual norm 4.717923953595e+03
  1 KSP Residual norm 7.893795860567e-11
t=-1.9999999999999991,Jt=137.6043,J=146.16566,Jt-J=-8.561360000000008,der=10.865566156960863,delta1=1.2884040893966615
t=-1.9999999999999991,Jt=137.6043,J=146.16566,Jt-J=-8.561360000000008,der=-8.307361999499339,delta2=-0.595169702179054
ff-mpirun -np 8 output/caseCylinderTest/solve.edp -v -1 -nw

4024.36user 8.31system 1:07:13elapsed 99%CPU (0avgtext+0avgdata 6962788maxresident)k
16inputs+71440outputs (1major+2739354minor)pagefaults 0swaps


 (17.16s)
FreeFem++ output/caseCylinderTest/interpolate_ns.edp -v -1 -nw (8.30s)
tau = 0.0158489, thetay(1,1,1) = -0.0112069
  0 KSP Residual norm 1.182396957645e+04
  1 KSP Residual norm 1.096533811116e-10
Text = 0, int2d = 132.351
  0 KSP Residual norm 4.676111636480e+03
  1 KSP Residual norm 3.284115939938e-11
t=-1.799999999999999,Jt=132.35112,J=146.16566,Jt-J=-13.814539999999994,der=17.220761834403138,delta1=1.491855973480987
t=-1.799999999999999,Jt=132.35112,J=146.16566,Jt-J=-13.814539999999994,der=-13.16628148031665,delta2=-0.1882517668422209
ff-mpirun -np 8 output/caseCylinderTest/solve.edp -v -1 -nw

4037.37user 8.50system 1:07:26elapsed 99%CPU (0avgtext+0avgdata 6962780maxresident)k
0inputs+71456outputs (1major+2739341minor)pagefaults 0swaps


 (16.44s)
FreeFem++ output/caseCylinderTest/interpolate_ns.edp -v -1 -nw (8.08s)
tau = 0.0251189, thetay(1,1,1) = -0.0177617


In [ ]:
plt.plot(np.log10(T),np.log10(delta1),'-o')
plt.xlabel('$t$')
plt.ylabel('$\delta(t)$')
plt.savefig(output+'/SurfaceShapeDerivativeValidation.png')

plt.figure()
plt.plot(np.log10(T),np.log10(delta2),'-o')
plt.xlabel('$t$')
plt.ylabel('$\delta(t)$')
plt.savefig(output+'/VolumeShapeDerivativeValidation.png')

In [ ]:
from sklearn.linear_model import LinearRegression

X = np.log10(T).reshape(-1, 1) 
y = np.log10(delta1)
reg = LinearRegression().fit(X, y)
order1 = reg.coef_[0]
print(f"order surface = {order1}")

X = np.log10(T).reshape(-1, 1) 
y = np.log10(delta2)
reg = LinearRegression().fit(X, y)
order2 = reg.coef_[0]
print(f"order volume = {order2}")